# Python -> LLVM compiler

In [2]:
%run ~/.jupyter/config.ipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import ast_to_llvm.python_to_core as ptc
import ast_to_llvm.core_to_llvm as ctl
import ast_to_llvm.type_infer as ti
from ast_to_llvm.toplevel import autojit

from ctypes import CFUNCTYPE, c_double, c_int

# Final

In [27]:
@autojit
def f(a, b):
    c = a + b + 3
    return c
f(2, 4)

9

# Unification

In [33]:
from unification import unify, var, reify

In [57]:
x, y = var("x"), var("y")
mgu = unify((1, x, y), (1, y, x))
print(mgu)

reify(x, mgu)

{~x: ~y}


~y

# tests

In [ ]:
func = """
def f(a, b):
    c = a + b
    return c
"""

In [ ]:
core = ptc.PythonVisitor()(func)

In [ ]:
ti.TypeInfer().visit(core)

In [ ]:
f = ctl.LLVMEmitter().visit(core)

In [ ]:
ctl.llvm_init()
engine = ctl.create_llvm_engine()
ctl.compile_code(engine, str(f))

In [ ]:
func_ptr = engine.get_function_address("f")
cfunc = CFUNCTYPE(c_int, c_int, c_int)(func_ptr)
res = cfunc(2, 5)
print("f(...) =", res)